In [1]:
from pynq import Overlay
import pynq.lib.dma
import time

In [2]:
ol = Overlay("design_1.bit")

In [3]:
#until including this cell: run always, even just with CFFI
mkl_gpio0 = ol.axi_gpio_0
mkl_gpio1 = ol.axi_gpio_1
mkl_div = ol.clk_wiz_0

In [9]:
XCLK_CLK_CFG_2 = 0x208
XCLK_CLK_CFG_23 = 0x25C

XGPIO_DATA0 = 0x0
XGPIO_DATA2 = 0x8

CLK_DIV = 0x100       #0x000, 0x100, 0x200, ...

#it does not speed up, slower!
VAL_32BIT_WR  = 0x8000000E | CLK_DIV
VAL_32BIT_WRn = 0x0000000E | CLK_DIV
VAL_24BIT_WR  = 0x8000001E | CLK_DIV
VAL_24BIT_WRn = 0x0000001E | CLK_DIV
VAL_TA_WR     = 0x8000002E | CLK_DIV
VAL_TA_WRn    = 0x0000002E | CLK_DIV
VAL_32BIT_RD  = 0x8000004E | CLK_DIV
VAL_32BIT_RDn = 0x0000004E | CLK_DIV

TRG_CNT = 0

def trigger_cnt(inc):
    global TRG_CNT;
    if inc :
        TRG_CNT = TRG_CNT + 1
    return (TRG_CNT & 0x3) << 30
    
def cnfg0(QSPIio0):
    return 0

def cnfg1(QSPIio1):
    return 0

def cnfgdiv(ClkDiv):
    ClkDiv.write(XCLK_CLK_CFG_2, 0x10)                  #0x10 = 25 MHz, 0x40 = 6.25 MHz
    ClkDiv.write(XCLK_CLK_CFG_23, 0x03)
    return 0

def QSPI_Write32bit(val):
    mkl_gpio0.write(XGPIO_DATA0, val)
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_32BIT_WR)          #start trigger, set nCS active low
    mkl_gpio1.write(XGPIO_DATA0, 0x0000010E | trigger_cnt(1))
    ##s = 0
    ##while s == 0:                                     #wait for start trigger done (bit 31 = 1)
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_32BIT_WRn)         #do no the 32bit transaction (release trigger)
    ##s = 0
    ##while s == 0:                                     #wait for shift done (bit 0 = 1)
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ##s = mkl_gpio0.read(XGPIO_DATA2)
    ##print(hex(s))

def QSPI_Write32bitLE(val):
    mkl_gpio0.write(XGPIO_DATA0, val)
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_32BIT_WR)          #start trigger, set nCS active low
    mkl_gpio1.write(XGPIO_DATA0, 0x0000018E | trigger_cnt(1))
    ##s = 0
    ##while s == 0:                                     #wait for start trigger done (bit 31 = 1)
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_32BIT_WRn)         #do no the 32bit transaction (release trigger)
    ##s = 0
    ##while s == 0:                                     #wait for shift done (bit 0 = 1)
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ##s = mkl_gpio0.read(XGPIO_DATA2)
    ##print(hex(s))
        
def QSPI_Write24bit(val):
    mkl_gpio0.write(XGPIO_DATA0, val)
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_24BIT_WR)  #set bit for 24bit word to send
    mkl_gpio1.write(XGPIO_DATA0, 0x0000011E | trigger_cnt(1))
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_24BIT_WRn)
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ##s = mkl_gpio0.read(XGPIO_DATA2)
    ##print(hex(s))
        
def QSPI_TurnAround():
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_TA_WR)   #set bit for 2bit turnaround
    mkl_gpio1.write(XGPIO_DATA0, 0x0000012E | trigger_cnt(1))
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_TA_WRn)
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ##s = mkl_gpio0.read(XGPIO_DATA2)
    ##print(hex(s))
    
def QSPI_Read32bit():
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_32BIT_RD)  #trigger a new read cycle
    mkl_gpio1.write(XGPIO_DATA0, 0x0000014E | trigger_cnt(1))
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_32BIT_RDn)
    #s = 0
    #while s == 0:
    #    s = mkl_gpio1.read(XGPIO_DATA2)
        ##print(hex(s))
    s = mkl_gpio0.read(XGPIO_DATA2)
    return s

def QSPI_Read32bitLE():
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_32BIT_RD)  #trigger a new read cycle
    mkl_gpio1.write(XGPIO_DATA0, 0x000001CE | trigger_cnt(1))
    ##s = 0
    ##while s == 0:
    ##    s = mkl_gpio1.read(XGPIO_DATA2)
    ##    print(hex(s))
    ####mkl_gpio1.write(XGPIO_DATA0, VAL_32BIT_RDn)
    #s = 0
    #while s == 0:
    #    s = mkl_gpio1.read(XGPIO_DATA2)
        ##print(hex(s))
    s = mkl_gpio0.read(XGPIO_DATA2)
    return s
    
def QSPI_Deselect():
    global TRG_CNT;
    mkl_gpio1.write(XGPIO_DATA0, 0x0000010F | trigger_cnt(0))  #release all nCS signals
    TRG_CNT = 0
    

In [5]:
cnfg0(mkl_gpio0)
cnfg1(mkl_gpio1)
cnfgdiv(mkl_div)

0

In [6]:
QSPI_Deselect()
QSPI_Write32bit(0x010100110)
QSPI_Write32bit(0x12345670)
QSPI_Write24bit(0xABCDEF00)        #is val << 8
QSPI_Write32bitLE(0x98765400)      #N-times
QSPI_Write32bitLE(0xACBD2013)
QSPI_Write32bitLE(0xBCDEF012)
QSPI_Deselect()

In [ ]:
QSPI_Write32bit(0x010100110)
QSPI_Write32bit(0x12345678)
QSPI_Write24bit(0xABCDFE00)        #is val << 8
QSPI_TurnAround()
v1 = QSPI_Read32bitLE()            #N-times
v2 = QSPI_Read32bitLE()
v3 = QSPI_Read32bitLE()
QSPI_Deselect()
print(hex(v1))
print(hex(v2))
print(hex(v3))

In [10]:
from cffi import FFI
import array

wrVal = array.array('I', [0x01011100, 0x11223344, 0x55667700, 0x98765432, 0x12345678, 0x11223344, 0x55667788])
#ALT is <<8, 24bit

ffi = FFI()
ffi.cdef("""unsigned int C_QSPI_Init(int);
            unsigned int C_QSPI_Write(unsigned int *, unsigned int);
            void C_QSPI_Deinit(void);
         """)
clib = ffi.dlopen("/home/xilinx/c_code/libGeneric_QSPI.so")
i = clib.C_QSPI_Init(2)      #3 is max! = 11.111 MHz, 0 = 33.3333 MHz, 1 = 20 MHz
#print(i)
i = clib.C_QSPI_Write(ffi.cast("unsigned int *", ffi.from_buffer(wrVal)), len(wrVal))

clib.C_QSPI_Deinit()
print(i)

1


In [ ]:
from cffi import FFI
import array

wrVal = array.array('I', [0x01011100, 0x11223344, 0x55667700])   #ALT is <<8, 24bit
rdVal = array.array('I', [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

ffi = FFI()
ffi.cdef("""unsigned int C_QSPI_Init(int);
            unsigned int C_QSPI_Read(unsigned int *, unsigned int *, unsigned int);
            void C_QSPI_Deinit(void);
        """)
clib = ffi.dlopen("/home/xilinx/c_code/libGeneric_QSPI.so")
i = clib.C_QSPI_Init(4)      #2 is max! = 16.667 MHz, 0 = 50 MHz - without flow control!
#16.667MHz is the maximum continous QSPI speed for our throughput on QSPI
#print(i)
i = clib.C_QSPI_Read(ffi.cast("unsigned int *", ffi.from_buffer(wrVal)), ffi.cast("unsigned int *", ffi.from_buffer(rdVal)), len(rdVal))
clib.C_QSPI_Deinit()
print(i)
print(rdVal)


In [ ]:
#change the PLL clock for AXI bus and PL - decreases the Rx sample delay!
core = pynq.ps.Clocks
print(core.cpu_mhz)
print(core.fclk0_mhz)
core.fclk0_mhz = 333.3
print(core.fclk0_mhz)